### Importando bibliotecas úteis

In [40]:
import pandas as pd
import numpy as np
from pulp import *

### Definindo as filiais

In [41]:
Filiais = ["Seattle", "San Francisco", "Las Vegas", "Phoenix", "Denver"]

# Como todos são iguais, precisa mesmo desse dict?
Estoque = {
            "Seattle": 175,
            "San Francisco": 175,
            "Las Vegas": 175, 
            "Phoenix": 175, 
            "Denver": 175
}

EntregaMinima = {
            "Seattle": 25,
            "San Francisco": 30,
            "Las Vegas": 30, 
            "Phoenix": 35, 
            "Denver": 25
}

### Definindo os contratos

In [42]:
Contratos = ["Washington", "Oregon", "California", "Idaho", "Nevada", "Montana", "Wyoming", "Arizona", "Utah", "Colorado"]

Demanda = {
            "Washington": 100,
            "Oregon": 65,
            "California": 100,
            "Idaho": 70,
            "Nevada": 120,
            "Montana": 60,
            "Wyoming": 75,
            "Arizona": 100,
            "Utah": 95,
            "Colorado": 85   
}

# Por enquanto, comentado para não deixar o problema complexo demais
MinDeFornecedores = {
            "Washington": 3,
            "Oregon": 2,
            "California": 3,
            "Idaho": 2,
            "Nevada": 3,
            "Montana": 2,
            "Wyoming": 2,
            "Arizona": 3,
            "Utah": 3,
            "Colorado": 3   
}

### Custos de entrega, em R$/unid., para cada contrato a partir de cada filial

In [43]:
custos = [
    # Washington, Oregon, California, Idaho, Nevada, Montana, Wyoming, Arizona, Utah, Colorado
    [10, 15, 10, 15, 20, 20, 20, 40, 10, 30], # Seattle
    [30, 15, 10, 20, 10, 20, 20, 30, 20, 30], # San Francisco
    [20, 10, 5, 15, 10, 15, 15, 10, 5, 5], # Las Vegas
    [40, 25, 15, 20, 10, 30, 30, 10, 15, 10], # Phoenix
    [30, 30, 25, 10, 5, 35, 35, 15, 5, 10] # Denver
]

# Transformando num dicionário:
custos = makeDict([Filiais, Contratos], custos, 0) # Filiais é o nome das rows, Contratos é o nome das columns, custos são os elementos das matrizes. 0 é o default

### Lista de Tuplas com todas as rotas possíveis

In [44]:
Rotas = [(f, c) for f in Filiais for c in Contratos]
# Por exemplo, haverá a rota (["Washington"]["Seattle"])

### Custo das Rotas

In [45]:
# vars significa variáveis
# 0: custo mínimo, None: (não há) custo máximo, LpInteger significa variáveis inteiras
vars = LpVariable.dicts("Rotas", (Filiais, Contratos), 0, None, LpInteger)

### Definindo o problema de otimização

In [46]:
prob = LpProblem("Problema da Distribuição de Produtos", LpMinimize)

c:\Python310\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


### Adicionando a função objetivo


In [47]:
prob += (
    lpSum([vars[f][c] * custos[f][c] for (f, c) in Rotas]),
    "Soma_dos_Custos_de_Transporte"
)

### Adicionando as restrições de estoque e demanda

In [48]:
for f in Filiais:
    prob += (
        lpSum([vars[f][c] for c in Contratos]) <= Estoque[f],
        "Soma_dos_Produtos_na_Filial_%s" % f,
    )

for c in Contratos:
    prob += (
        lpSum([vars[f][c] for f in Filiais]) >= Demanda[c],
        "Soma_dos_Produtos_no_Contrato_%s" % c,
    )

### Adicionando restrição de Número Mínimo de Fornecedores

In [49]:
# for c in Contratos:
#     # print(len([vars[f][c].varValue for f in Filiais if vars[f][c].varValue > 0]))
#     print([vars[f][c].varValue ])

In [50]:
# for c in Contratos:
#     prob += (
#         len([vars[f][c].varValue for f in Filiais if vars[f][c].varValue > 0]) 
#                                                         >= MinDeFornecedores[c],
#         "Numero_min_de_Fornecedores_%s" %c
#     )

### Resolvendo o Problema

In [51]:
prob.solve()

print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

Status: Optimal
Rotas_Denver_Arizona = 0.0
Rotas_Denver_California = 0.0
Rotas_Denver_Colorado = 0.0
Rotas_Denver_Idaho = 70.0
Rotas_Denver_Montana = 0.0
Rotas_Denver_Nevada = 35.0
Rotas_Denver_Oregon = 0.0
Rotas_Denver_Utah = 70.0
Rotas_Denver_Washington = 0.0
Rotas_Denver_Wyoming = 0.0
Rotas_Las_Vegas_Arizona = 0.0
Rotas_Las_Vegas_California = 0.0
Rotas_Las_Vegas_Colorado = 85.0
Rotas_Las_Vegas_Idaho = 0.0
Rotas_Las_Vegas_Montana = 0.0
Rotas_Las_Vegas_Nevada = 0.0
Rotas_Las_Vegas_Oregon = 65.0
Rotas_Las_Vegas_Utah = 25.0
Rotas_Las_Vegas_Washington = 0.0
Rotas_Las_Vegas_Wyoming = 0.0
Rotas_Phoenix_Arizona = 100.0
Rotas_Phoenix_California = 0.0
Rotas_Phoenix_Colorado = 0.0
Rotas_Phoenix_Idaho = 0.0
Rotas_Phoenix_Montana = 0.0
Rotas_Phoenix_Nevada = 75.0
Rotas_Phoenix_Oregon = 0.0
Rotas_Phoenix_Utah = 0.0
Rotas_Phoenix_Washington = 0.0
Rotas_Phoenix_Wyoming = 0.0
Rotas_San_Francisco_Arizona = 0.0
Rotas_San_Francisco_California = 25.0
Rotas_San_Francisco_Colorado = 0.0
Rotas_San_Francisc

### Organizando em forma de Dataframe

In [52]:
data_array = []
for v in prob.variables():
    data_array.append(v.varValue)

data_array = np.array(data_array).reshape(5, 10)

data_array

array([[  0.,   0.,   0.,  70.,   0.,  35.,   0.,  70.,   0.,   0.],
       [  0.,   0.,  85.,   0.,   0.,   0.,  65.,  25.,   0.,   0.],
       [100.,   0.,   0.,   0.,   0.,  75.,   0.,   0.,   0.,   0.],
       [  0.,  25.,   0.,   0.,  60.,  10.,   0.,   0.,   0.,  75.],
       [  0.,  75.,   0.,   0.,   0.,   0.,   0.,   0., 100.,   0.]])

### Organizando Filiais e Contratos em ordem alfabética, para coincidir com os resultados apresentados
Os resultados foram apresentados em ordem alfabética

In [53]:
Filiais.sort()
Contratos.sort()

In [54]:
df = pd.DataFrame(data = data_array, index = Filiais, columns = Contratos)

df

,Arizona,California,Colorado,Idaho,Montana,Nevada,Oregon,Utah,Washington,Wyoming
Denver,0.0,0.0,0.0,70.0,0.0,35.0,0.0,70.0,0.0,0.0
Las Vegas,0.0,0.0,85.0,0.0,0.0,0.0,65.0,25.0,0.0,0.0
Phoenix,100.0,0.0,0.0,0.0,0.0,75.0,0.0,0.0,0.0,0.0
San Francisco,0.0,25.0,0.0,0.0,60.0,10.0,0.0,0.0,0.0,75.0
Seattle,0.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0


### Adicionando linha de custo

In [55]:
custos_individuais = {}
for c in Contratos:
    # custos_individuais.append(lpSum([vars[f][c].varValue * custos[f][c] for f in Filiais]))
    custos_individuais[c] = lpSum([vars[f][c].varValue * custos[f][c] for f in Filiais])
    
custos_individuais

{'Arizona': 1000.0,
 'California': 1000.0,
 'Colorado': 425.0,
 'Idaho': 700.0,
 'Montana': 1200.0,
 'Nevada': 1025.0,
 'Oregon': 650.0,
 'Utah': 475.0,
 'Washington': 1000.0,
 'Wyoming': 1500.0}

### Finalmente, para o custo total:

In [56]:
print("Custo total dos Transportes = R${:.2f}".format(value(prob.objective)))

Custo total dos Transportes = R$8975.00
